In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported, UnslothTrainer, UnslothTrainingArguments, FastLanguageModel
from datasets import load_dataset, DatasetDict, Dataset
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import os

In [ ]:
max_seq_length = 2048 # Choose any. Unsloth support RoPE Scaling internally
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,
)

## creating dataset B

In [ ]:
dataset_whole = load_dataset('theatticusproject/cuad-qa', trust_remote_code=True)

df_train = pd.DataFrame(dataset_whole['train'])
df_test = pd.DataFrame(dataset_whole['test'])

df_combined = pd.concat([df_train, df_test])

df_cuad_combined = df_combined.drop_duplicates(subset='context')

dataset_cuad =  Dataset.from_pandas(df_cuad_combined)

In [ ]:
with open('nli_dataset/dev.json', 'r') as file:
    data = json.load(file)

documents = data['documents']

rows = []
for doc in documents:
    row = {
        'id': doc['id'],
        'file_name': doc['file_name'],
        'text': doc['text'],
        'url': doc['url']
    }
    rows.append(row)
    
df_dev = pd.DataFrame(rows)

with open('nli_dataset/test.json', 'r') as file:
    data = json.load(file)

documents = data['documents']

rows = []
for doc in documents:
    row = {
        'id': doc['id'],
        'file_name': doc['file_name'],
        'text': doc['text'],
        'url': doc['url']
    }
    rows.append(row)
    
df_test = pd.DataFrame(rows)

with open('nli_dataset/train.json', 'r') as file:
    data = json.load(file)

documents = data['documents']

rows = []
for doc in documents:
    row = {
        'id': doc['id'],
        'file_name': doc['file_name'],
        'text': doc['text'],
        'url': doc['url']
    }
    rows.append(row)
    
df_train = pd.DataFrame(rows)

df_nli_combined = pd.concat([df_dev,df_test,df_train])

dataset_nli = Dataset.from_pandas(df_nli_combined)

In [ ]:
folder_path = 'source_documents'
file_contents_list = []

file_ids = []
file_contents = []

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Open the file and read its contents into a string
        with open(file_path, 'r') as file:
            file_contents.append(file.read())
        
        # Append the file name to the list
        file_ids.append(filename)

# Create a DataFrame from the lists
df = pd.DataFrame({'id': file_ids, 'contract': file_contents})
df['source'] = "Genie"
df_genie = df

dataset_genie = Dataset.from_pandas(df_genie)

In [ ]:
# Extract and rename columns from df_cuad_combined
df_cuad = df_cuad_combined[['id', 'context']].rename(columns={'context': 'contract'})
df_cuad['source'] = 'CUAD'

In [ ]:
# Extract and rename columns from df_nli_combined
df_nli = df_nli_combined[['id', 'text']].rename(columns={'text': 'contract'})
df_nli['source'] = 'NLI'

In [ ]:
# Combine the DataFrames
df_combined = pd.concat([df_cuad, df_nli, df_genie], ignore_index=True)
# make sure id type is consistent
df_combined['id'] = df_combined['id'].astype(str)

In [ ]:
dataset_combined = Dataset.from_pandas(df_combined)

In [ ]:
dataset_combined

## training

In [ ]:
trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    #train_dataset = dataset,
    train_dataset = dataset_combined,
    #dataset_text_field = "text",
    dataset_text_field = 'contract',
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_ratio = 0.1,
        #max_steps = 2000,
        #max_steps = 5,
        num_train_epochs = 1,

        # Select a 2 to 10x smaller learning rate for the embedding matrices!
        learning_rate = 5e-6,
        embedding_learning_rate = 1e-6,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        #save_steps = 100,
        save_steps = 50,
        save_total_limit = 10,
        optim = "adamw_8bit",
        weight_decay = 0.00,
        lr_scheduler_type = "cosine",
        seed = 3407,
        #output_dir = "./drive/MyDrive/Llama-3-8B-fineweb-edu-r128a32wd0lstcosinelr5e06-10BT",
        output_dir = 'results' # not sure?
    ),
)

trainer_stats = trainer.train()

In [ ]:
# saving the LoRA adapters locally
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

In [ ]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model_1_epoch_B", tokenizer,)
# add ModelFile after this

In [ ]:
!ollama create unsloth_model_1_epoch_B -f ./model_1_epoch_B/Modelfile

In [ ]:
import openai
import os

os.environ["OPENAI_API_KEY"] = "hellooooo"

In [ ]:
# change the file path to the last checkpoint
file_path = 'results/checkpoint-72/trainer_state.json'
with open(file_path, 'r') as file:
    data = json.load(file)

log_history = data['log_history']
steps = [entry['step'] for entry in log_history]
loss = [entry['loss'] for entry in log_history]

#plt.figure(figsize=(10, 6))
plt.plot(steps, loss, linestyle='-', color='purple')
plt.xlabel('Step')
plt.ylabel('Loss')
plt.title('Training Loss over Steps for 1 Epoch, Dataset B')
plt.savefig('Training Loss over Steps for 1 Epoch, Dataset B', format = 'pdf')
plt.show()